# Loading code and data

## When loading notebook for the first time:

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
!mkdir /root/.kaggle

In [ ]:
########################################
########## Add kaggle API key ##########
########################################

!echo '{"username":"","key":""}' > ~/.kaggle/kaggle.json

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!git clone https://github.com/TigerManon/drive-on-mars.git

In [ ]:
%cd drive-on-mars/
%mkdir raw_data
%cd raw_data

In [ ]:
!kaggle datasets download yash92328/ai4mars-terrainaware-autonomous-driving-on-mars

In [ ]:
!unzip -q ai4mars-terrainaware-autonomous-driving-on-mars.zip

In [ ]:
!rm ai4mars-terrainaware-autonomous-driving-on-mars.zip

In [ ]:
%cd ../

In [ ]:
%ls

## When reloading the notebook

In [ ]:
%ls

In [ ]:
%pwd

In [ ]:
%cd drive-on-mars/

# Loading Libraries


In [ ]:
#!pip install git+https://github.com/qubvel/segmentation_models
!pip install segmentation_models

In [ ]:
# Data Visualisation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tqdm import tqdm


from tensorflow.keras import callbacks
from sklearn.model_selection import train_test_split


In [ ]:
# Segmentation Models

############# Alice #################
# Had to add this to make sm work in colab
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
from tensorflow import keras
import segmentation_models as sm
#####################################

import segmentation_models as sm
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf


# DeepLabV3_ResNet50 Model Tuning


## Data Generator for DeepLab_Res50_Model

In [ ]:
# 0 - soil
# 1 - bedrock
# 2 - sand
# 3 - big rock
# 255 -> 4 - NULL (no label)

In [ ]:
# Define a customized data generator
class UnetDataGenerator(keras.utils.Sequence):
    def __init__(self, image_folder, label_folder, input_shape, batch_size, num_classes, subfolder, split_percent, use_mask = False):
        self.image_folder       = image_folder
        self.label_folder       = label_folder
        self.input_shape        = input_shape
        self.batch_size         = batch_size
        self.num_classes        = num_classes
        self.subfolder          = subfolder
        self.split_percent      = split_percent
        self.path_df            = self.make_df()
        self.split_df()
        self.use_mask           = use_mask

    def __len__(self):
        print("Length of generator:", len(self.path_df))
        return math.ceil(len(self.path_df) / float(self.batch_size))    
    
    def make_df(self):      
        img_list = [f for f in os.listdir(self.image_folder) if f.endswith('.JPG')]
        lab_list = [f for f in os.listdir(self.label_folder) if f.endswith('.png')]

        
        path_df = pd.DataFrame(columns=["image_path", "label_path", "rover_mask_path", "range_mask_path"])

        for label_name in lab_list:
            if self.subfolder in ("train", "val"):
                image_name = label_name.replace('.png', '.JPG')
                rover_mask_name = label_name.replace('EDR', 'MXY')  
                range_mask_name = label_name.replace('EDR', 'RNG')  
                
            elif self.subfolder == 'test':
                image_name = label_name.replace('_merged.png', '.JPG')
                rover_mask_name = label_name.replace('_merged.png', '.png').replace('EDR', 'MXY')   
                range_mask_name = label_name.replace('_merged.png', '.png').replace('EDR', 'RNG')  
            
            parent_folder = os.path.dirname(self.image_folder)
            rover_mask_path = os.path.join(parent_folder,'mxy', rover_mask_name)
            range_mask_path = os.path.join(parent_folder,'rng-30m', range_mask_name)

            if image_name in img_list and os.path.exists(rover_mask_path) and os.path.exists(range_mask_path):
                new_row = pd.DataFrame([{
                "image_path": os.path.join(self.image_folder, image_name),
                "label_path": os.path.join(self.label_folder, label_name),
                "rover_mask_path": rover_mask_path,
                "range_mask_path": range_mask_path
                }])
                path_df = pd.concat([path_df, new_row], ignore_index=True)
        
        return path_df

  
    
    def split_df(self):
        if self.subfolder in ["train","test"]:
            self.path_df = self.path_df.iloc[:int(len(self.path_df) * self.split_percent)]
        elif self.subfolder == "val":
            self.path_df = self.path_df.iloc[int(len(self.path_df) * self.split_percent):]
    
    
    def __getitem__(self, index):
        input_images  = []
        output_targets = []

        #-------------------------------#
        #   Calculate start indice and end indice of the batch
        #-------------------------------#  
        start = index * self.batch_size
        end = min((index + 1) * self.batch_size, len(self.path_df)) # Make sure that we can load all the data of the last batch

        for i in range(start, end):  
            
            #-------------------------------#
            #   Get image,label,rover mask and range mask path of each batch from path_df
            #-------------------------------#                     
            jpg = self.path_df.iloc[i]["image_path"]
            png = self.path_df.iloc[i]["label_path"]
            rover = self.path_df.iloc[i]["rover_mask_path"]
            rng = self.path_df.iloc[i]["range_mask_path"]
            
            #-------------------------------#
            #   Load both range and rover masks combined
            #-------------------------------#           
            rover_array = cv2.imread(rover)
            rng_array = cv2.imread(rng)
            # reversing mask to only keep the image out of the mask
            mask_combined = (1-rover_array) * (1-rng_array)
      
            #-------------------------------#
            #   Transform images and labels to numpy array, resize them
            #   Set the background label to 4
            #-------------------------------#

            jpg = cv2.imread(jpg)
            if self.use_mask:
                jpg = jpg * mask_combined            
            
            png = cv2.imread(png,0)
                
            jpg = cv2.resize(jpg, dsize = (int(self.input_shape[0]), int(self.input_shape[1])))
            png = cv2.resize(png, dsize = (int(self.input_shape[0]), int(self.input_shape[1])), 
                                      interpolation = cv2.INTER_NEAREST)

            png[png == 255] = 4

            #-------------------------------------------------------#
            #   One hot encode the labels
            #-------------------------------------------------------#
                
            seg_labels = np.eye(self.num_classes)[png.reshape([-1])]                
            seg_labels = seg_labels.reshape((int(self.input_shape[0]), int(self.input_shape[1]), self.num_classes))

            input_images.append(jpg)
            output_targets.append(seg_labels)

        input_images = np.asarray(input_images)
        output_targets = np.array(output_targets)

        
        return input_images, output_targets


In [ ]:
# image and label directories paths in COLAB

# image_folder = "raw_data/ai4mars-dataset-merged-0.1/msl/images/edr"
# label_folder = "raw_data/ai4mars-dataset-merged-0.1/msl/labels/train"

In [ ]:
# image and label directories paths in LOCAL
image_folder = "/home/gargantua/code/TigerManon/08-Palette/ai4mars-dataset-merged-0.1/msl/images/edr"
label_folder = "/home/gargantua/code/TigerManon/08-Palette/ai4mars-dataset-merged-0.1/msl/labels/train"

In [ ]:
# Some inputs
batch_size = 16
num_classes = 5
input_shape = (256,256,3)
split_percent = 0.8
height=input_shape[0]
width=input_shape[1]
channels=input_shape[2]

In [ ]:
%%time
# Train dataset generator and validation dataset generator
traingen = UnetDataGenerator(image_folder,
                             label_folder,
                             input_shape,
                             batch_size,
                             num_classes,
                             "train",
                             split_percent,
                            use_mask = True)

valgen = UnetDataGenerator(image_folder,
                           label_folder,
                           input_shape,
                           batch_size,
                           num_classes,
                           "val",
                           split_percent,
                          use_mask = True)

## DeepLab_ResNet50 Model Initialization


In [ ]:
NUM_CLASSES = 5
IMAGE_SIZE = 256

In [ ]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=(3, 3),
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]),
        interpolation="nearest",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [ ]:
# Use a ResNet50 pretrained on ImageNet as the backbone model
def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    preprocessed = preprocess_input(model_input)
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=preprocessed
    )

    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="nearest",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="nearest",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)


deep_model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
deep_model.summary()

In [ ]:
# loss: dice_loss
categorical_focal_loss = CategoricalFocalLoss()
dice_loss = DiceLoss()
categorical_focal_dice_loss = categorical_focal_loss + dice_loss
# categorical_focal_jaccard_loss = categorical_focal_loss + jaccard_loss
# iou_score = IOUScore() (threshold: value to round predictions (use ``>`` comparison), if ``None`` prediction will not be round, as: metrics = [sm.metrics.IOUScore(threshold=0.5)])
# class_indexes: Optional integer or list of integers, classes to consider, if ``None`` all classes are used.
metrics=['accuracy', iou_score]

deep_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss=categorical_focal_dice_loss,
    metrics=metrics,
)

## DeepLab_ResNet50 Model Training

In [ ]:
%%time
MODEL = "DeepLab_Res50_ImageNet_img_256pix"

modelCheckpoint = callbacks.ModelCheckpoint("{}.h5".format(MODEL),
                                            monitor="val_iou_score",
                                            save_best_only = False)

LR_reducer = callbacks.ReduceLROnPlateau(patience = 3,
                                         monitor="val_iou_score",
                                         factor = 0.1,
                                         min_lr = 0
                                        )

early_stopper = callbacks.EarlyStopping(patience = 5,
                                        monitor="val_iou_score",
                                        restore_best_weights=True)



deep_history = deep_model.fit(traingen,
          validation_data=valgen,
          epochs=100,
          callbacks = [modelCheckpoint, LR_reducer, early_stopper],
          verbose=1)


In [ ]:
deep_history.history

In [ ]:
def plot_history(deep_history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(deep_history.history['loss'], label = 'train' + exp_name)
    ax1.plot(deep_history.history['val_loss'], label = 'val' + exp_name)
    ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(deep_history.history['iou_score'], label='train iou'  + exp_name)
    ax2.plot(deep_history.history['val_iou_score'], label='val iou'  + exp_name)
    ax2.set_ylim(0, 0.7)
    ax2.set_title('iou')
    ax2.legend()
    plt.show()

plot_history(deep_history)

## DeepLab_ResNet50 Model Saving

In [ ]:
!mkdir raw_data/models
!ls raw_data

In [ ]:
deep_model.save(f'raw_data/models/{MODEL}.h5')

In [ ]:
!ls raw_data/models

In [ ]:

%cp raw_data/models/DeepLab_Res50_ImageNet_img_256pix.h5 /content/drive/MyDrive/data/models

## Training Records

09/12/2023 23:34




*   Model: unet_4deep_16in_256pix
*   optimizer: 'adam', loss: sm.losses.bce_jaccard_loss, metrics: sm.metrics.
    iou_score
*   Trainable params: 1941173
*   Data: 12851 train images; 3213 val images, loaded,preporcessed,splited by   
      UnetDataGenerator
*   Training: set up for 100 epochs, stopped at 6 epochs by early stopping,
    Wall time: 29min 52s, GPU V100
*   'loss': 0.5861780643463135,
    'iou_score': 0.5779913663864136,
    'val_loss':  0.5868656635284424],
    'val_iou_score': 0.5775318145751953,
    'lr': [0.001, 0.001, 0.001, 0.001, 0.000100000005, 0.000100000005]





10/12/2023 03:34

* Model: unet_4deep_16in_1024pix

* optimizer: 'adam', loss: sm.losses.bce_jaccard_loss, metrics: sm.metrics.
  iou_score
* Trainable params: 1941173
* Data: image and label size: 1024, 12851 train images; 3213 val images, loaded, preporcessed,splited by UnetDataGenerator
* Training: set up for 100 epochs, stopped at 6 epochs by early stopping,        Wall time: 2h 8min 6s, GPU V100
* 'loss':  0.6727234125137329, 'iou_score': 0.50568687915802, 'val_loss': 0.6704270839691162, 'val_iou_score': 0.5068952441215515, 'lr': [0.001, 0.001, 0.001, 0.001, 0.000100000005, 0.000100000005]




10/12/2023 10:34

* Model: unet_4deep_64in_256pix

* optimizer: 'adam', loss: sm.losses.bce_jaccard_loss, metrics: sm.metrics. iou_score

* Trainable params: 31032005

* Data: image and label size: 256, 12851 train images; 3213 val images, loaded, preporcessed,splited by UnetDataGenerator

* Training: set up for 100 epochs, stopped at 6 epochs by early stopping, Wall time: 29min 3s, GPU A100

* 'loss': 0.5815900564193726, 'iou_score': 0.580341637134552, 'val_loss': 0.59281826019287112, 'val_iou_score': 0.5689619183540344, 'lr': [0.001, 0.001, 0.001, 0.001, 0.000100000005, 0.000100000005] Slightly overfitted

10/12/2023 16:34

* Model: Unet_ResBackBone_ImageNet_img_256pix

* optimizer: 'adam', loss: sm.losses.bce_jaccard_loss, metrics: sm.metrics. iou_score

* Total params: 24456734 (93.30 MB)
Trainable params: 24439384 (93.23 MB)
Non-trainable params: 17350 (67.77 KB)

* Data: image and label size: 256, 12851 train images; 3213 val images, loaded, preporcessed,splited by UnetDataGenerator

* Training: set up for 100 epochs, stopped at 6 epochs by early stopping, Wall time: 27min 3s, GPU A100

* 'loss': 0.4621942937374115
 'accuracy': 0.8795599341392517
 'iou_score': 0.659989595413208
 'val_loss': 0.48931652307510376
 'val_accuracy':  0.8699352741241455
 'val_iou_score': 0.6389757394790649
 'lr': [0.001, 0.001, 0.001, 0.001, 0.000100000005, 0.000100000005]}


10/12/2023 18:34

* Model: Unet_ResBackBone_ImageNet_img_256pix

* optimizer: 'adam', loss:'categorical_crossentropy', metrics: accuracy, iou_score

* Total params: 24456734 (93.30 MB)
Trainable params: 24439384 (93.23 MB)
Non-trainable params: 17350 (67.77 KB)

* Data: image and label size: 256, 12851 train images; 3213 val images, loaded, preporcessed,splited by UnetDataGenerator

* Training: set up for 100 epochs, stopped at 6 epochs by early stopping, Wall time: 30min, GPU A100

* 'loss': 0.28132763504981995
 'accuracy': 0.8859129548072815
 'iou_score': 0.5797572731971741
 'val_loss': 0.31182384490966797
 'val_accuracy':  0.8733996152877808
 'val_iou_score': 0.5677776336669922
 'lr': [0.001, 0.001, 0.001, 0.001, 0.000100000005, 0.000100000005]


## History

In [ ]:
# %%time
# es = callbacks.EarlyStopping(patience=2,restore_best_weights=True)
# #modelCheckpoint = callbacks.ModelCheckpoint("{}.h5".format('First_model'), monitor="val_loss", verbose=0, save_best_only=False)


# history = model.fit(X_train, y_train,
#           batch_size=16,
#           epochs=10,
#           validation_split=0.3,
#           callbacks=[es],
#           verbose=1)


In [ ]:
# plot_history(history)

In [ ]:
%%time
es = callbacks.EarlyStopping(patience=2,restore_best_weights=True)
#modelCheckpoint = callbacks.ModelCheckpoint("{}.h5".format('First_model'), monitor="val_loss", verbose=0, save_best_only=False)


history = model.fit(X_train, y_train,
          batch_size=16,
          epochs=100,
          validation_split=0.3,
          callbacks=[es],
          verbose=1)

In [ ]:
history.history

In [ ]:
# def plot_history(history, title='', axs=None, exp_name=""):
#     if axs is not None:
#         ax1, ax2 = axs
#     else:
#         f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

#     if len(exp_name) > 0 and exp_name[0] != '_':
#         exp_name = '_' + exp_name
#     ax1.plot(history.history['loss'], label = 'train' + exp_name)
#     ax1.plot(history.history['val_loss'], label = 'val' + exp_name)
#     ax1.set_ylim(0., 2.2)
#     ax1.set_title('loss')
#     ax1.legend()

#     ax2.plot(history.history['iou_score'], label='train iou'  + exp_name)
#     ax2.plot(history.history['val_iou_score'], label='val iou'  + exp_name)
#     ax2.set_ylim(0, 0.5)
#     ax2.set_title('iou')
#     ax2.legend()
#     return (ax1, ax2)

# plot_history(history)
# plt.show

In [ ]:
print(1)